# 20 뉴스 그룹 분류

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all',random_state=156)

### 데이터 탐색

In [3]:
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
print('target 클래스의 값과 분포도')
print(pd.Series(news_data.target).value_counts().sort_index())

target 클래스의 값과 분포도
0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [5]:
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

### 훈련/테스트용 데이터 추출

In [7]:
train_news = fetch_20newsgroups(subset='train',random_state=156,
                                remove=('headers', 'footers','quotes'))
X_train = train_news.data
y_train = train_news.target

In [8]:
train_news.data[0]

"\n\nWhat I did NOT get with my drive (CD300i) is the System Install CD you\nlisted as #1.  Any ideas about how I can get one?  I bought my IIvx 8/120\nfrom Direct Express in Chicago (no complaints at all -- good price & good\nservice).\n\nBTW, I've heard that the System Install CD can be used to boot the mac;\nhowever, my drive will NOT accept a CD caddy is the machine is off.  How can\nyou boot with it then?\n\n--Dave\n"

In [9]:
train_news.target[0]

4

In [10]:
test_news = fetch_20newsgroups(subset='test',random_state=156,
                                remove=('headers', 'footers','quotes'))
X_test = test_news.data
y_test = test_news.target

In [11]:
len(X_train),len(X_test)

(11314, 7532)

### 피쳐 벡터화 변환과 머신러닝 모델 학습/예측/평가
- Case 1. CountVectorizer

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(X_train)
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

In [13]:
X_train_count.shape

(11314, 101631)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_count,y_train)

LogisticRegression()

In [15]:
pred = lr_clf.predict(X_test_count)
accuracy_score(y_test, pred)

0.6055496548061604

In [16]:
print(y_test[:5])
print(pred[:5])

[ 4 11  1  7  8]
[ 4 11  6  7  8]


- TfidVectorizer

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)

In [18]:
X_train_tfidf.shape

(11314, 101631)

In [19]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf,y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.6736590546999469

In [20]:
print(y_test[:5])
print(pred[:5])

[ 4 11  1  7  8]
[5 1 1 7 8]


### 학습에 사용된 파라미터

In [21]:
print(tfidf_vect.get_params())

{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


In [22]:
print(lr_clf.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


- Case 3. stop_words filtering, ngram (1, 2), max_df = 300 이라는 조건으로 실행

In [23]:
tfidf_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)

In [24]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf,y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.6922464152947424

- Case 4. Case 3에서 LogisticRegression C값을 10으로

In [26]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf,y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.7010090281465746

### Pipeline 과 GridSearchCV 를 통한 하이퍼 파라미터 튜닝

In [27]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf',LogisticRegression())
])

In [28]:
params = {
    'tfidf_vect__ngram_range' : [(1,2)],
    'tfidf_vect__max_df': [300, 700],
    'lr_clf__C': [1,10]
}

In [30]:
from sklearn.model_selection import GridSearchCV

grid_pipe = GridSearchCV(pipeline, param_grid=params, scoring='accuracy',verbose=1,n_jobs=-1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_,grid_pipe.best_score_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 